### **Phase 2: Ball Tracking and Player Statistics with DEEPSORT** 🎥⚽


1. **Player & Ball Tracking** ⚽

In [ ]:
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load YOLO model
model = YOLO("/Users/alyazouzou/Desktop/CV_Football/FootCVision2/phase1/runs/detect/train/weights/best.pt")

# Initialize Deep SORT
tracker = DeepSort(max_age=30, n_init=3, nn_budget=100)

# Video paths
input_video_path = "/Users/alyazouzou/Desktop/CV_Football/vids/short.mov"
output_video_path = "/Users/alyazouzou/Desktop/CV_Football/vids/demo_tracked.mov"

# Open video
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writer
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection with YOLO
    results = model(frame, verbose=False)[0]
    
    # Extract bounding boxes, confidences, and class IDs
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy.numpy().flatten()
        conf = box.conf.numpy().item()
        class_id = int(box.cls.numpy().item())
        detections.append([[x1, y1, x2 - x1, y2 - y1], conf, class_id])

    # Perform tracking with Deep SORT
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracked objects on the frame
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()  # Get bounding box (left, top, right, bottom)
        x1, y1, x2, y2 = map(int, ltrb)

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.waitKey(int(1000 / fps))  # Adjust frame delay to match FPS

    # Write the frame to the output video
    out.write(frame)

    # Display the frame (optional)
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

2025-01-05 22:48:28.747 python[3884:126565] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-05 22:48:28.747 python[3884:126565] +[IMKInputSession subclass]: chose IMKInputSession_Modern


2. **Statistics** 📈
   - Extract metrics for each player, such as:
     - Distance covered.
     - Ball possession time.
     - Speed and acceleration.